### CoreBx_island - Interpolate the North Core Banks DEMs onto rotated 1-m grid and save in a .nc file.

May 1, 2022 - Second round of refactoring

#### Input:
* YAML file with island box params `small_island_box.yml`
* DSM tiffs in published format
* DSM tiffs that were clipped a second time by Jin-Si to remove sandbars
* DSM tiffs from 2019-09 lidar

#### Output: in `/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/`:
* .nc file with first four published maps as array `[name]_pub.nc`
* .nc file with first four re-clipped maps as array `[name]_reclip.nc`
* .nc file with two lidar maps (ground and canopy) `[name]_lidar.nc`

[name] is provided in the dict loadted from `small_island_box.yml` - Currently `ncorebx_small`

TODO: Add better metatdata to the NC files

#### Notes:
 - The reclipped files have funny names because it took several iterations of exporting from GM before their bounds were big enough.  
 - The lidar data were processed by Andy. `ground` is 50th percentile of points classified ground. `canopy` is 90th percentile of 1st return points.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray
from scipy import interpolate
from CoreBx_funcs import *
%matplotlib inline

In [2]:
drv, computername = which_computer()
print('Working on',computername)
print('drv:',drv)

Working on IGSAGIEGLTCHS10
drv: C:/


### Read in a dict to define the rotated coordinate system

In [3]:
r = yaml2dict('small_island_box.yml')
print(r)

# Make a grid 
xu,yu,xrot,yrot,xcoords,ycoords = make_grid(**r)

ny,nx = np.shape(xu)
print('Size of grid:',ny,nx)

{'name': 'ncorebx_small', 'e0': 383520.0, 'n0': 3860830.0, 'xlen': 25000.0, 'ylen': 1200.0, 'dxdy': 1.0, 'theta': 42.0}
make_grid: Shape of xrot, yrot:  (1200, 25000) (1200, 25000)
corners x, corners y]
[[ 383520.03700711 3860830.70613772]
 [ 402097.91449922 3877558.30216608]
 [ 401295.62690219 3878449.33281183]
 [ 382717.74941009 3861721.73678346]
 [ 383520.03700711 3860830.70613772]]
Saving to ncorebx_small.csv
Size of grid: 1200 25000


In [4]:
# Input folders
pub_dir = drv+'/crs/proj/2019_DorianOBX/Best_files/dems/'
reclip_dir = drv+'/crs/proj/2019_DorianOBX/Best_files/2022-04-22_nosandbars/'
lidar_dir = drv+'/crs/proj/2019_DorianOBX/Best_files/lidar/'

# output folder for rotated DEMs
dem_path =drv+'/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/'

# file names for reclipped DSMs (no sandbars)
fnames_reclip = (\
           pub_dir+'/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif',\
           reclip_dir+'/NCB_2019-09-12-13_clipped_cog_r2.tif',\
           reclip_dir+'/NCB_2019-10-11_clipped_cog_reshape.tif',\
           reclip_dir+'/NCB_2019-11-26_clipped_cog_r3.tif')

# output file for array of reclipped DSMs
reclip_nc = dem_path+r['name']+'_reclip.nc'

# file names for published DSMs
fnames_pub = (\
           pub_dir+'/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif',\
           pub_dir+'/20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif',\
           pub_dir+'/20191011_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18N_1m_cog.tif',\
           pub_dir+'/20191126_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18N_1m_cog.tif')

# output file for array of published DSMs
pub_nc = dem_path+r['name']+'_pub.nc'

# file names for lidar
fnames_lidar = (\
               lidar_dir+'2019_NCMP_PostDorian_CoBa_UTM18_gnd50_UTM18_1m_DSM_cog.tif',\
               lidar_dir+'2019_NCMP_PostDorian_CoBa_UTM18_1st90_UTM18_1m_DSM_cog.tif')

# output file for array of lidar DSMs
lidar_nc = dem_path+r['name']+'_lidar.nc'

titles = ([\
         "Aug 30 2019 pre-Dorian",\
         "Sep 12-13 2019 post-Dorian",\
         "Oct 11 2019",\
         "Nov 26 2019 post-Nor'easter"])

dates = ([\
         "2019-08-30",\
         "2019-09-12",\
         "2019-10-11",\
         "2019-11-26"])

nf = len(fnames_reclip)
nft = len(titles)
print('Length of datasets',nf,nft)

Length of datasets 4 4


### Rotate reclipped maps to array, save to .nc

In [5]:
%%time
dslist=[]
for i, fn in enumerate(fnames_reclip):
    iswarned = False
    fpath = fn
    print(i, fpath)

    da = rioxarray.open_rasterio( fpath, masked=True )

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))
        
    # this is the fast iteration, which only works when all of the source points fall inside the target box
    try:
        zi=f((yu,xu))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    except:
        if(not iswarned):
            print("Warning: using slow iteration.")
            iswarned = True
        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

    #dar = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    dar = xr.DataArray(zi,dims=['Cross-shore','Alongshore'],coords={'Cross-shore': ycoords, 'Alongshore' :xcoords })

    dar = dar.chunk()
    dslist.append(dar)

dsa = xr.concat(dslist, dim='map')

# TODO - Add some metadata to this netcdf file. Add time to the maps. Are the dimensions in the right order?

dsa.to_netcdf(reclip_nc)

0 C://crs/proj/2019_DorianOBX/Best_files/dems//20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif
(24189,) (27433,) (1, 24189, 27433)
(27433,) (24189,) (24189, 27433)
1 C://crs/proj/2019_DorianOBX/Best_files/2022-04-22_nosandbars//NCB_2019-09-12-13_clipped_cog_r2.tif
(18803,) (21255,) (1, 18803, 21255)
(21255,) (18803,) (18803, 21255)
2 C://crs/proj/2019_DorianOBX/Best_files/2022-04-22_nosandbars//NCB_2019-10-11_clipped_cog_reshape.tif
(19100,) (20940,) (1, 19100, 20940)
(20940,) (19100,) (19100, 20940)
3 C://crs/proj/2019_DorianOBX/Best_files/2022-04-22_nosandbars//NCB_2019-11-26_clipped_cog_r3.tif
(19791,) (22379,) (1, 19791, 22379)
(22379,) (19791,) (19791, 22379)
CPU times: total: 36.7 s
Wall time: 49.4 s


### Rotate published maps to array, save as .nc

In [6]:
%%time
dslist=[]
for i, fn in enumerate(fnames_pub):
    iswarned = False
    fpath = fn
    print(i, fpath)

    da = rioxarray.open_rasterio( fpath, masked=True )

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))
        
    # this is the fast iteration, which only works when all of the source points fall inside the target box
    try:
        zi=f((yu,xu))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    except:
        if(not iswarned):
            print("Warning: using slow iteration.")
            iswarned = True
        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

    #dar = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    dar = xr.DataArray(zi,dims=['Cross-shore','Alongshore'],coords={'Cross-shore': ycoords, 'Alongshore' :xcoords })

    dar = dar.chunk()
    dslist.append(dar)

dsa = xr.concat(dslist, dim='map')

# TODO - Add some metadata to this netcdf file. Add time to the maps. Are the dimensions in the right order?

dsa.to_netcdf(pub_nc)

0 C://crs/proj/2019_DorianOBX/Best_files/dems//20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif
(24189,) (27433,) (1, 24189, 27433)
(27433,) (24189,) (24189, 27433)
1 C://crs/proj/2019_DorianOBX/Best_files/dems//20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif
(23899,) (27276,) (1, 23899, 27276)
(27276,) (23899,) (23899, 27276)
2 C://crs/proj/2019_DorianOBX/Best_files/dems//20191011_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18N_1m_cog.tif
(52966,) (46193,) (1, 52966, 46193)
(46193,) (52966,) (52966, 46193)
3 C://crs/proj/2019_DorianOBX/Best_files/dems//20191126_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18N_1m_cog.tif
(55497,) (47653,) (1, 55497, 47653)
(47653,) (55497,) (55497, 47653)
CPU times: total: 2min 2s
Wall time: 4min 29s


### Rotate lidar maps to array, save as .nc

In [7]:
%%time
dslist=[]
for i, fn in enumerate(fnames_lidar):
    iswarned = False
    fpath = fn
    print(i, fpath)

    da = rioxarray.open_rasterio( fpath, masked=True )

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))
        
    # this is the fast iteration, which only works when all of the source points fall inside the target box
    try:
        zi=f((yu,xu))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    except:
        if(not iswarned):
            print("Warning: using slow iteration.")
            iswarned = True
        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

    #dar = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    dar = xr.DataArray(zi,dims=['Cross-shore','Alongshore'],coords={'Cross-shore': ycoords, 'Alongshore' :xcoords })

    dar = dar.chunk()
    dslist.append(dar)

dsa = xr.concat(dslist, dim='map')

# TODO - Add some metadata to this netcdf file. Add time to the maps. Are the dimensions in the right order?

dsa.to_netcdf(lidar_nc)

0 C://crs/proj/2019_DorianOBX/Best_files/lidar/2019_NCMP_PostDorian_CoBa_UTM18_gnd50_UTM18_1m_DSM_cog.tif
(58539,) (50750,) (1, 58539, 50750)
(50750,) (58539,) (58539, 50750)
1 C://crs/proj/2019_DorianOBX/Best_files/lidar/2019_NCMP_PostDorian_CoBa_UTM18_1st90_UTM18_1m_DSM_cog.tif
(58539,) (50750,) (1, 58539, 50750)
(50750,) (58539,) (58539, 50750)
CPU times: total: 4min 41s
Wall time: 4min 58s
